In [3]:
import numpy as np
np.random.seed(4)
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader.data as web
import datetime

ModuleNotFoundError: No module named 'pandas_datareader'

Resuelto por Manuel Arturo Díaz Salazar

In [4]:
pip install pandas_datareader

Note: you may need to restart the kernel to use updated packages.


In [5]:
from sklearn.preprocessing import MinMaxScaler
from keras.layers import Dense, LSTM
from keras.models import Sequential

ImportError: Keras requires TensorFlow 2.2 or higher. Install TensorFlow via `pip install tensorflow`

In [ ]:
#
# Funciones auxiliares
#
def graficar_predicciones(real, prediccion):
    plt.plot(real[0:len(prediccion)],color='red', label='Valor real de la acción')
    plt.plot(prediccion, color='blue', label='Predicción de la acción')
    plt.ylim(1.1 * np.min(prediccion)/2, 1.1 * np.max(prediccion))
    plt.xlabel('Tiempo')
    plt.ylabel('Valor de la acción')
    plt.legend()
    plt.show()

In [ ]:
#
# Lectura de los datos
#
#dataset = pd.read_csv('Futuros del Índice S&P 500 - Este es el bueno.csv', index_col='Fecha', parse_dates=['Fecha'])
#dataset.head()
start = datetime.datetime(2010,1,1)
end = datetime.datetime(2020,12,31)
dataset = web.get_data_yahoo('VOO','01/01/2010',interval='m')
dataset.head()

In [ ]:
#
# Sets de entrenamiento y validación 
# La LSTM se entrenará con datos de 2016 hacia atrás. La validación se hará con datos de 2017 en adelante.
# En ambos casos sólo se usará el valor más alto de la acción para cada día
#

set_entrenamiento = dataset['2010':'2019'].iloc[:,3:4]
set_validacion = dataset['2020':].iloc[:,3:4]
set_entrenamiento['Close'].plot(legend=True)
set_validacion['Close'].plot(legend=True)
plt.legend(['Entrenamiento (Ene 2010-Dic 2019)', 'Validacion (Ene 2020 - Actualidad)'])
plt.show()

In [ ]:
# Normalización del set de entrenamiento
sc = MinMaxScaler(feature_range=(0,1))
print(sc)
print(set_entrenamiento)
set_entrenamiento_escalado = sc.fit_transform(set_entrenamiento)

## La red LSTM tendrá como entrada "time_step" datos consecutivos, y como salida 1 dato (la predicción a
# partir de esos "time_step" datos). Se conformará de esta forma el set de entrenamiento
time_step = 726
X_train = []
Y_train = []
m = len(set_entrenamiento_escalado)

In [ ]:
time_step = 3
X_train = [] 
Y_train = [] 
m = len(set_entrenamiento_escalado)

In [ ]:
for i in range(time_step,m):
    # X: bloques de "time_step" datos: 0-time_step, 1-time_step+1, 2-time_step+2, etc
    X_train.append(set_entrenamiento_escalado[i-time_step:i,0])
    # Y: el siguiente dato
    Y_train.append(set_entrenamiento_escalado[i,0])
X_train, Y_train = np.array(X_train), np.array(Y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
len(X_train)

In [ ]:
#
# Red LSTM
#
dim_entrada = (X_train.shape[1],1)
dim_salida = 1
print(dim_entrada)
na = 120

modelo = Sequential()
modelo.add(LSTM(units=na, input_shape=dim_entrada))
modelo.add(Dense(units=dim_salida))
modelo.compile(optimizer='rmsprop', loss='mse')
modelo.fit(X_train,Y_train,epochs=20,batch_size=12)

In [ ]:
#
# Validación (predicción del valor de las acciones)
#

x_test = set_validacion.values
x_test = sc.transform(x_test)

X_test = []

m = len(x_test)
for i in range(time_step,m):
  X_test.append(x_test[i-time_step:i,0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))

prediccion = modelo.predict(X_test)
prediccion = sc.inverse_transform(prediccion)

# Graficar resultados
print(set_validacion)
print(prediccion)
graficar_predicciones(set_validacion.values,prediccion)